In [ ]:
import pandas as pd

In [2]:
merged = pd.read_csv('merged_dataset.csv')

C:\Users\PC\AppData\Local\Temp\ipykernel_2460\3613210809.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv('merged_dataset.csv')


In [3]:
new_merged = merged.iloc[:, :-3].dropna().sort_values(by='Date')
new_merged

,Season,Wk,Date,Home,Away,Home_Goals,Away_Goals,Result,League
84685,1888-1889,1.0,1888-09-08,Preston,Burnley,5.0,2.0,1.0,Premier-League
84686,1888-1889,1.0,1888-09-08,Everton,Accrington,2.0,1.0,1.0,Premier-League
84687,1888-1889,1.0,1888-09-08,Wolves,Aston Villa,1.0,1.0,0.5,Premier-League
84688,1888-1889,1.0,1888-09-08,Bolton,Derby County,3.0,6.0,0.0,Premier-League
84689,1888-1889,1.0,1888-09-08,Stoke,West Brom,0.0,2.0,0.0,Premier-League
...,...,...,...,...,...,...,...,...,...
172340,2024-2025,25.0,2025-02-22,St. Gallen,Basel,2.0,2.0,0.5,Swiss-Super-League
147841,2024-2025,27.0,2025-02-22,Aberdeen,Kilmarnock,1.0,0.0,1.0,Scottish-Premiership
172339,2024-2025,25.0,2025-02-22,Zürich,Yverdon-Sport,2.0,1.0,1.0,Swiss-Super-League
174627,2024-2025,5.0,2025-04-16,Kryvbas Kryvyi Rih,Shakhtar,0.0,1.0,0.0,Ukrainian-Premier-League


In [18]:
import pandas as pd

# Load the merged dataset
df = new_merged.copy()

# Initialize Elo ratings
elo_ratings = {}
INITIAL_ELO = 1500
K = 40  # Elo K-factor for football

# Function to calculate the expected score
def expected_score(elo_team, elo_opponent):
    return 1 / (1 + 10 ** ((elo_opponent - elo_team) / 400))

# Function to update Elo ratings
def update_elo(old_elo, opponent_elo, result):
    expected = expected_score(old_elo, opponent_elo)
    return old_elo + K * (result - expected)

# Lists to store Elo ratings for each match
home_elo_list = []
away_elo_list = []

# Iterate over matches in chronological order
for index, row in df.iterrows():
    home_team = row["Home"]
    away_team = row["Away"]
    
    # Get current Elo ratings or assign default
    home_elo = elo_ratings.get(home_team, INITIAL_ELO)
    away_elo = elo_ratings.get(away_team, INITIAL_ELO)
    
    # Store Elo ratings before the match
    home_elo_list.append(home_elo)
    away_elo_list.append(away_elo)

    # Determine match result
    if row["Home_Goals"] > row["Away_Goals"]:
        home_result, away_result = 1, 0  # Home win
    elif row["Home_Goals"] < row["Away_Goals"]:
        home_result, away_result = 0, 1  # Away win
    else:
        home_result, away_result = 0.5, 0.5  # Draw

    # Update Elo ratings
    new_home_elo = update_elo(home_elo, away_elo, home_result)
    new_away_elo = update_elo(away_elo, home_elo, away_result)

    # Save new ratings
    elo_ratings[home_team] = new_home_elo
    elo_ratings[away_team] = new_away_elo

# Add Elo ratings to the DataFrame
df["Home_Elo"] = home_elo_list
df["Away_Elo"] = away_elo_list

# Display the updated DataFrame
print(df.head())


          Season   Wk        Date     Home          Away  Home_Goals  \
84685  1888-1889  1.0  1888-09-08  Preston       Burnley         5.0   
84686  1888-1889  1.0  1888-09-08  Everton    Accrington         2.0   
84687  1888-1889  1.0  1888-09-08   Wolves   Aston Villa         1.0   
84688  1888-1889  1.0  1888-09-08   Bolton  Derby County         3.0   
84689  1888-1889  1.0  1888-09-08    Stoke     West Brom         0.0   

       Away_Goals  Result          League  Home_Elo  Away_Elo  
84685         2.0     1.0  Premier-League    1500.0    1500.0  
84686         1.0     1.0  Premier-League    1500.0    1500.0  
84687         1.0     0.5  Premier-League    1500.0    1500.0  
84688         6.0     0.0  Premier-League    1500.0    1500.0  
84689         2.0     0.0  Premier-League    1500.0    1500.0  


In [24]:
# Convert Elo ratings dictionary to a sorted DataFrame
elo_ranking = pd.DataFrame(elo_ratings.items(), columns=["Team", "Elo_Rating"])
elo_ranking = elo_ranking.sort_values(by="Elo_Rating", ascending=False).reset_index(drop=True)

# Display the top teams
elo_ranking.head(20)

,Team,Elo_Rating
0,Arsenal,1965.832388
1,Liverpool,1950.629458
2,Leverkusen,1939.329422
3,Bayern Munich,1920.231992
4,Paris S-G,1918.500839
5,Inter,1902.959232
6,Galatasaray,1896.133447
7,Real Madrid,1886.484914
8,Atlético Madrid,1882.268979
9,Barcelona,1877.754404


In [ ]:
import pandas as pd
import numpy as np
import math

def calculate_elo_ratings(df):
    """
    Calculate Elo ratings for football teams, with competition-specific coefficients.
    
    Args:
        df: DataFrame with match data
    
    Returns:
        DataFrame with added Elo ratings and competition coefficients
    """
    # Create a copy of the dataframe to avoid modifying the original
    df_elo = df.copy()
    
    # Constants for Elo calculation
    K = 20  # K-factor determines how much ratings change after each match
    BASE_ELO = 1500  # Initial Elo rating for teams
    
    # Dictionaries to store team Elo ratings and match counts
    team_elo = {}  # Current Elo rating for each team
    team_matches = {}  # Number of matches played by each team
    
    # Dictionary to store competition-specific Elo ratings
    competition_team_elo = {}  # Structure: {league: {team: elo}}
    
    # Add Elo columns to the dataframe
    df_elo['home_elo'] = 0.0
    df_elo['away_elo'] = 0.0
    df_elo['home_elo_after'] = 0.0
    df_elo['away_elo_after'] = 0.0
    df_elo['comp_coef'] = 1.0  # Default coefficient
    
    # Process each match chronologically
    for i, row in df_elo.sort_values(by=['Date']).iterrows():
        home_team = row['Home']
        away_team = row['Away']
        league = row['League']
        
        # Initialize team Elo if not already present
        if home_team not in team_elo:
            team_elo[home_team] = BASE_ELO
            team_matches[home_team] = 0
        
        if away_team not in team_elo:
            team_elo[away_team] = BASE_ELO
            team_matches[away_team] = 0
        
        # Initialize competition-specific Elo for teams if needed
        if league not in competition_team_elo:
            competition_team_elo[league] = {}
        
        if home_team not in competition_team_elo[league]:
            competition_team_elo[league][home_team] = BASE_ELO
        
        if away_team not in competition_team_elo[league]:
            competition_team_elo[league][away_team] = BASE_ELO
        
        # Record pre-match Elo ratings
        df_elo.at[i, 'home_elo'] = team_elo[home_team]
        df_elo.at[i, 'away_elo'] = team_elo[away_team]
        
        # Calculate expected outcome
        home_win_expectancy = 1 / (1 + 10 ** ((team_elo[away_team] - team_elo[home_team]) / 400))
        away_win_expectancy = 1 - home_win_expectancy
        
        # Determine actual outcome (1 for home win, 0.5 for draw, 0 for away win)
        if row['Result'] == 1.0:  # Home win
            actual_outcome = 1.0
        elif row['Result'] == 0.0:  # Draw
            actual_outcome = 0.5
        else:  # Away win
            actual_outcome = 0.0
        
        # Calculate K-factor adjustment based on goal difference and match importance
        goal_diff = abs(row['Home_Goals'] - row['Away_Goals'])
        k_multiplier = math.log(goal_diff + 1) * 1.2 if goal_diff > 0 else 1
        
        # Adjust K-factor for teams with few matches (new teams have more volatile ratings)
        home_k = K * k_multiplier * (2.2 / (1 + 0.1 * team_matches[home_team])) if team_matches[home_team] < 30 else K * k_multiplier
        away_k = K * k_multiplier * (2.2 / (1 + 0.1 * team_matches[away_team])) if team_matches[away_team] < 30 else K * k_multiplier
        
        # Calculate competition coefficient if we have enough data
        comp_coefs = {}
        if sum(len(teams) for teams in competition_team_elo.values()) > 10:
            league_avg_elos = {}
            for comp, teams in competition_team_elo.items():
                if len(teams) > 3:  # Only consider leagues with at least 3 teams
                    league_avg_elos[comp] = sum(teams.values()) / len(teams)
            
            if league_avg_elos:
                # Standardize the league average Elos
                mean_elo = sum(league_avg_elos.values()) / len(league_avg_elos)
                std_elo = np.std(list(league_avg_elos.values())) if len(league_avg_elos) > 1 else 1
                
                for comp, avg_elo in league_avg_elos.items():
                    # Calculate standardized value and add 1 to avoid multiplying by 0
                    comp_coefs[comp] = 1 + (avg_elo - mean_elo) / (std_elo if std_elo > 0 else 1) * 0.2
        
        # Use competition coefficient if available, otherwise default to 1.0
        comp_coef = comp_coefs.get(league, 1.0) if comp_coefs else 1.0
        df_elo.at[i, 'comp_coef'] = comp_coef
        
        # Update Elo ratings based on match outcome
        elo_change_home = home_k * comp_coef * (actual_outcome - home_win_expectancy)
        elo_change_away = away_k * comp_coef * (away_win_expectancy - actual_outcome)
        
        # Update global team Elo ratings
        team_elo[home_team] += elo_change_home
        team_elo[away_team] += elo_change_away
        
        # Update competition-specific Elo ratings
        competition_team_elo[league][home_team] += elo_change_home * 1.2  # Higher weight for competition-specific ratings
        competition_team_elo[league][away_team] += elo_change_away * 1.2
        
        # Record post-match Elo ratings
        df_elo.at[i, 'home_elo_after'] = team_elo[home_team]
        df_elo.at[i, 'away_elo_after'] = team_elo[away_team]
        
        # Increment match count for teams
        team_matches[home_team] += 1
        team_matches[away_team] += 1
    
    return df_elo

# Apply the function to your dataframe
new_merged_with_elo = calculate_elo_ratings(new_merged)

In [5]:
new_merged_with_elo

,Season,Wk,Date,Home,Away,Home_Goals,Away_Goals,Result,League,home_elo,away_elo,home_elo_after,away_elo_after,comp_coef
84685,1888-1889,1.0,1888-09-08,Preston,Burnley,5.0,2.0,1.0,Premier-League,1500.000000,1500.000000,1536.598171,1463.401829,1.000000
84686,1888-1889,1.0,1888-09-08,Everton,Accrington,2.0,1.0,1.0,Premier-League,1500.000000,1500.000000,1518.299086,1481.700914,1.000000
84687,1888-1889,1.0,1888-09-08,Wolves,Aston Villa,1.0,1.0,0.5,Premier-League,1500.000000,1500.000000,1478.000000,1522.000000,1.000000
84688,1888-1889,1.0,1888-09-08,Bolton,Derby County,3.0,6.0,0.0,Premier-League,1500.000000,1500.000000,1500.000000,1500.000000,1.000000
84689,1888-1889,1.0,1888-09-08,Stoke,West Brom,0.0,2.0,0.0,Premier-League,1500.000000,1500.000000,1500.000000,1500.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172340,2024-2025,25.0,2025-02-22,St. Gallen,Basel,2.0,2.0,0.5,Swiss-Super-League,1442.471763,2352.454697,1442.354355,2374.567979,1.111534
147841,2024-2025,27.0,2025-02-22,Aberdeen,Kilmarnock,1.0,0.0,1.0,Scottish-Premiership,1565.208940,1337.993451,1568.654641,1325.249143,0.973219
172339,2024-2025,25.0,2025-02-22,Zürich,Yverdon-Sport,2.0,1.0,1.0,Swiss-Super-League,1071.939479,1417.139193,1088.136151,1414.918817,1.107091
174627,2024-2025,5.0,2025-04-16,Kryvbas Kryvyi Rih,Shakhtar,0.0,1.0,0.0,Ukrainian-Premier-League,1461.454720,2885.580754,1470.980753,2895.106787,1.145894


In [ ]:
import pandas as pd

def get_team_elo_rankings(df):
    """
    Calculate the latest Elo rating for each team and display them in ranked order.
    
    Args:
        df: DataFrame with match data including Elo ratings
        
    Returns:
        DataFrame with teams and their latest Elo ratings, sorted from highest to lowest
    """
    # Ensure the dataframe has the necessary Elo columns
    if 'home_elo_after' not in df.columns or 'away_elo_after' not in df.columns:
        raise ValueError("DataFrame must contain 'home_elo_after' and 'away_elo_after' columns")
    
    # Sort by date to ensure we're processing matches chronologically
    df_sorted = df.sort_values(by=['Date'])
    
    # Dictionary to store the latest Elo rating for each team
    latest_elo = {}
    
    # Process each match to update the latest Elo ratings
    for _, row in df_sorted.iterrows():
        home_team = row['Home']
        away_team = row['Away']
        
        latest_elo[home_team] = row['home_elo_after']
        latest_elo[away_team] = row['away_elo_after']
    
    # Create a DataFrame from the dictionary
    rankings = pd.DataFrame({
        'Team': list(latest_elo.keys()),
        'Elo Rating': list(latest_elo.values())
    })
    
    # Sort teams by Elo rating (descending)
    rankings = rankings.sort_values(by='Elo Rating', ascending=False).reset_index(drop=True)
    
    # Add rank column
    rankings.insert(0, 'Rank', rankings.index + 1)
    
    # Format Elo ratings to 1 decimal place
    rankings['Elo Rating'] = rankings['Elo Rating'].round(1)
    
    return rankings

def get_league_rankings(df, league_name):
    """
    Get team rankings for a specific league only
    
    Args:
        df: DataFrame with match data including Elo ratings
        league_name: Name of the league to filter by
        
    Returns:
        DataFrame with teams and their latest Elo ratings for the specified league
    """
    # Filter the dataframe to only include matches from the specified league
    df_league = df[df['League'] == league_name]
    
    # Get all teams that participated in this league
    league_teams = set(pd.concat([df_league['Home'], df_league['Away']]).unique())
    
    # Get rankings for all teams
    all_rankings = get_team_elo_rankings(df)
    
    # Filter to only include teams from the specified league
    league_rankings = all_rankings[all_rankings['Team'].isin(league_teams)].reset_index(drop=True)
    league_rankings['Rank'] = league_rankings.index + 1
    
    return league_rankings

def get_competition_strength_ranking(df):
    """
    Calculate average Elo rating for each competition/league
    
    Args:
        df: DataFrame with match data including Elo ratings
        
    Returns:
        DataFrame with leagues and their average team Elo ratings
    """
    # Get team rankings
    team_rankings = get_team_elo_rankings(df)
    
    # Create a dictionary to store teams by league
    league_teams = {}
    for _, row in df.iterrows():
        league = row['League']
        home_team = row['Home']
        away_team = row['Away']
        
        if league not in league_teams:
            league_teams[league] = set()
        
        league_teams[league].add(home_team)
        league_teams[league].add(away_team)
    
    # Calculate average Elo for each league
    league_avg_elo = {}
    for league, teams in league_teams.items():
        league_team_elos = team_rankings[team_rankings['Team'].isin(teams)]['Elo Rating']
        league_avg_elo[league] = league_team_elos.mean()
    
    # Create a DataFrame from the dictionary
    league_rankings = pd.DataFrame({
        'League': list(league_avg_elo.keys()),
        'Average Elo': list(league_avg_elo.values())
    })
    
    # Sort leagues by average Elo rating (descending)
    league_rankings = league_rankings.sort_values(by='Average Elo', ascending=False).reset_index(drop=True)
    
    # Add rank column
    league_rankings.insert(0, 'Rank', league_rankings.index + 1)
    
    # Format average Elo ratings to 1 decimal place
    league_rankings['Average Elo'] = league_rankings['Average Elo'].round(1)
    
    return league_rankings

# Example usage:
# First, calculate Elo ratings if not already done
# new_merged_with_elo = calculate_elo_ratings(new_merged)

# Then get team rankings
team_rankings = get_team_elo_rankings(new_merged_with_elo)
print(team_rankings.head(50))  # Show top 20 teams

# To get ranking of competitions by average team strength
# competition_rankings = get_competition_strength_ranking(new_merged_with_elo)
# print(competition_rankings)

    Rank             Team  Elo Rating
0      1        Liverpool      7511.9
1      2    Bayern Munich      6807.7
2      3        Barcelona      6796.9
3      4      Real Madrid      6557.7
4      5   Manchester Utd      6490.2
5      6          Arsenal      5861.1
6      7         Juventus      5759.2
7      8            Porto      5469.0
8      9            Milan      4886.3
9     10          Benfica      4678.3
10    11        Paris S-G      4616.3
11    12            Inter      4598.4
12    13    PSV Eindhoven      4504.5
13    14             Ajax      4345.0
14    15      Sporting CP      4253.6
15    16        Tottenham      4098.3
16    17             Roma      4063.2
17    18         Dortmund      3962.5
18    19             Lyon      3813.7
19    20       Leverkusen      3646.7
20    21  Manchester City      3474.9
21    22  Atlético Madrid      3467.0
22    23          Everton      3418.9
23    24      RB Salzburg      3407.2
24    25        Feyenoord      3319.9
25    26    

In [9]:
# To get Premier League rankings only
premier_league_rankings = get_league_rankings(new_merged_with_elo, 'La-Liga')
print(premier_league_rankings.head(20))

    Rank             Team  Elo Rating
0      1        Barcelona      6796.9
1      2      Real Madrid      6557.7
2      3  Atlético Madrid      3467.0
3      4         Valencia      3112.3
4      5          Sevilla      2452.0
5      6       Villarreal      2042.0
6      7    Dep La Coruña      1941.2
7      8        La Coruña      1809.6
8      9    Athletic Club      1583.0
9     10           Girona      1527.3
10    11            Xerez      1430.6
11    12      Real Burgos      1426.0
12    13          Leganés      1394.6
13    14            Eibar      1388.7
14    15          Córdoba      1385.9
15    16       Compostela      1385.6
16    17        Gimnàstic      1381.8
17    18    Real Sociedad      1369.2
18    19           Huesca      1365.9
19    20      Extremadura      1365.3
